In [1]:
import deepchem as dc
from deepchem.feat.graph_data import GraphData
from deepchem.feat import MolGraphConvFeaturizer
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from deepchem.models.torch_models import TorchModel

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (C:\Users\yyyyx\miniconda3\envs\deepchem\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [23]:
from deepchem.models.optimizers import Adam

In [2]:
df = pd.read_csv('./AqsolDB.csv')

In [3]:
df

,Unnamed: 0,Drug_ID,Drug,Y
0,0,"N,N,N-trimethyloctadecan-1-aminium bromide",CCCCCCCCCCCCCCCCCC[N+](C)(C)C.[Br-],-3.616127
1,1,Benzo[cd]indol-2(1H)-one,O=C1Nc2cccc3cccc1c23,-3.254767
2,2,4-chlorobenzaldehyde,O=Cc1ccc(Cl)cc1,-2.177078
3,3,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",CC(c1ccccc1)c1cc(C(=O)[O-])c(O)c(C(C)c2ccccc2)...,-3.924409
4,4,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,c1cc(N(CC2CO2)CC2CO2)ccc1Cc1ccc(N(CC2CO2)CC2CO...,-4.662065
...,...,...,...,...
9977,9977,tetracaine,CCCCNc1ccc(C(=O)OCCN(C)C)cc1,-3.010000
9978,9978,tetracycline,CN(C)[C@@H]1C(O)=C(C(N)=O)C(=O)[C@@]2(O)C(O)=C...,-2.930000
9979,9979,thymol,Cc1ccc(C(C)C)c(O)c1,-2.190000
9980,9980,verapamil,COc1ccc(CCN(C)CCCC(C#N)(c2ccc(OC)c(OC)c2)C(C)C...,-3.980000


In [25]:
# 1. SMILES에서 그래프 형식으로 변환
featurizer = dc.feat.ConvMolFeaturizer()
features = featurizer.featurize(df["Drug"])  # Smiles 컬럼에서 특징 추출

Failed to featurize datapoint 4792, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(class RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True, bool includeAtomMaps=True, bool includeChiralPresence=False)
Failed to featurize datapoint 5043, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(class RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True, bool includeAtomMaps=True, bool includeChiralPresence=False)
Exception message: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (9982,) + inhomogeneous part.


In [ ]:
GraphConv
features = featurizer.featurize()
(df["Drug"])

In [26]:
features

array([<deepchem.feat.mol_graphs.ConvMol object at 0x0000022523D36B20>,
       ...,
      dtype=object)

In [27]:
# 2. 레이블 설정 (pIC50)
labels = df["Y"].values

In [28]:
# 3. 가중치 (필요시 기본값으로 1 설정)
weights = None  # 기본적으로 None으로 설정. 커스텀 가중치가 있으면 지정.

# 4. 데이터셋 생성
dataset = dc.data.NumpyDataset(X=features, y=labels, w=weights)

In [29]:
invalid_indices = []
for i, x_item in enumerate(dataset.X):
    # 예: x_item 이 비어 있거나, 그래프가 아닌 ndarray인 경우 invalid 처리
    if (isinstance(x_item, np.ndarray) and len(x_item) == 0):
        invalid_indices.append(i)

print("Invalid indices:", invalid_indices)


Invalid indices: [4792, 5043]


In [30]:
import deepchem as dc
import numpy as np

# 유효한 인덱스를 골라낸다
valid_indices = [
    i for i in range(len(dataset.X)) if i not in invalid_indices
]

# 유효한 X, y, w, ids만 추출
filtered_X   = [dataset.X[i] for i in valid_indices]
filtered_y   = [dataset.y[i] for i in valid_indices]
filtered_w   = [dataset.w[i] for i in valid_indices]
filtered_ids = [dataset.ids[i] for i in valid_indices]

# 새롭게 NumpyDataset 생성
new_dataset = dc.data.NumpyDataset(
    X=filtered_X,
    y=filtered_y,
    w=filtered_w,
    ids=filtered_ids
)

# 이제 new_test_dataset을 대신 사용하면 됨


In [31]:
dataset

<NumpyDataset X.shape: (9982,), y.shape: (9982,), w.shape: (9982,), task_names: [0]>

In [50]:
new_dataset_pytorch = new_dataset.make_pytorch_dataset()

In [51]:
# 데이터셋 확인
print(f"Number of samples in dataset: {len(new_dataset)}")
print(f"Feature shape: {new_dataset.X[0].node_features.shape[0]} nodes, {new_dataset.X[0].edge_index.shape[1]} edges")
print(f"First label: {new_dataset.y[0]}")

TypeError: object of type '_TorchNumpyDataset' has no len()

In [34]:
# 데이터셋을 Train/Validation/Test로 나누기
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(new_dataset)

# 확인
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(valid_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 7984
Validation dataset size: 998
Test dataset size: 998


In [54]:
train_dataset = train_dataset.make_pytorch_dataset
test_dataset = test_dataset.make_pytorch_dataset
valid_dataset = valid_dataset.make_pytorch_dataset

In [55]:
# GPU 또는 CPU 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [56]:
# 하이퍼파라미터 목록 정의
param_grid = {
    'graph_conv_layers': [[64, 64], [128, 128], [64, 128, 64]],
    'dense_layer_size': [128, 256],
    'dropout': [0.2, 0.3],
    'learning_rate': [1e-3, 5e-4, 1e-4],
    'batch_size': [128]
}

In [19]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from itertools import product
from tqdm import tqdm
import deepchem as dc
from deepchem.models.torch_models import TorchModel

# PyTorch 기반 GCN 모델 정의
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout):
        super(GCN, self).__init__()
        layers = []
        for i in range(len(hidden_dims)):
            in_dim = input_dim if i == 0 else hidden_dims[i - 1]
            layers.append(nn.Linear(in_dim, hidden_dims[i]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        layers.append(nn.Linear(hidden_dims[-1], output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# 하이퍼파라미터 조합 반복
combinations = list(product(
    param_grid['graph_conv_layers'],
    param_grid['dense_layer_size'],
    param_grid['dropout'],
    param_grid['learning_rate'],
    param_grid['batch_size']
))

# 결과를 저장할 데이터프레임 초기화
results = []

for i, (graph_conv_layers, dense_layer_size, dropout, learning_rate, batch_size) in enumerate(tqdm(combinations, desc="Grid Search Progress")):
    print(f"\nRunning combination {i+1}/{len(combinations)}:")
    print(f"graph_conv_layers: {graph_conv_layers}, dense_layer_size: {dense_layer_size}, dropout: {dropout}, learning_rate: {learning_rate}, batch_size: {batch_size}")

    # PyTorch 모델 초기화
    input_dim = 75  # ConvMolFeaturizer의 기본 출력 크기
    output_dim = 1
    torch_model = GCN(input_dim, graph_conv_layers, output_dim, dropout).to(device)
    loss = nn.MSELoss()
    # DeepChem 옵티마이저 사용
    dc_optimizer = Adam(learning_rate=learning_rate)

    # DeepChem의 TorchModel로 래핑
    try:
        model = TorchModel(torch_model, loss=loss, optimizer=dc_optimizer, batch_size=batch_size, device=device)

        # 모델 학습
        model.fit(train_dataset, nb_epoch=100)

        # 평가
        metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
        train_score = model.evaluate(train_dataset, [metric], transformers)
        valid_score = model.evaluate(valid_dataset, [metric], transformers)

        # 결과 저장
        results.append({
            'graph_conv_layers': graph_conv_layers,
            'dense_layer_size': dense_layer_size,
            'dropout': dropout,
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'train_r2_score': train_score.get('pearson_r2_score', None),
            'valid_r2_score': valid_score.get('pearson_r2_score', None)
        })

        print(f"Train R2 Score: {train_score.get('pearson_r2_score', 'N/A')}, Validation R2 Score: {valid_score.get('pearson_r2_score', 'N/A')}")
    except Exception as e:
        print(f"Error during training: {e}")

# 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과가 있는 경우만 저장 및 출력
if not results_df.empty:
    results_df = results_df.sort_values(by='valid_r2_score', ascending=False)
    results_df.to_csv("gridsearch_results_pytorch.csv", index=False)
    print("\nTop Results:")
    print(results_df.head())
else:
    print("No valid results collected.")


Grid Search Progress: 100%|███████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 461.53it/s]


Running combination 1/36:
graph_conv_layers: [64, 64], dense_layer_size: 128, dropout: 0.2, learning_rate: 0.001, batch_size: 128
Error during training: 'Adam' object has no attribute '_create_pytorch_optimizer'

Running combination 2/36:
graph_conv_layers: [64, 64], dense_layer_size: 128, dropout: 0.2, learning_rate: 0.0005, batch_size: 128
Error during training: 'Adam' object has no attribute '_create_pytorch_optimizer'

Running combination 3/36:
graph_conv_layers: [64, 64], dense_layer_size: 128, dropout: 0.2, learning_rate: 0.0001, batch_size: 128
Error during training: 'Adam' object has no attribute '_create_pytorch_optimizer'

Running combination 4/36:
graph_conv_layers: [64, 64], dense_layer_size: 128, dropout: 0.3, learning_rate: 0.001, batch_size: 128
Error during training: 'Adam' object has no attribute '_create_pytorch_optimizer'

Running combination 5/36:
graph_conv_layers: [64, 64], dense_layer_size: 128, dropout: 0.3, learning_rate: 0.0005, batch_size: 128
Error during t

In [58]:
dc_optimizer = Adam(learning_rate=learning_rate)
model = TorchModel(torch_model,loss = 0.0, device=device) #, loss=loss, optimizer=dc_optimizer, batch_size=batch_size

# 모델 학습
model.fit(train_dataset, nb_epoch=100)

TypeError: __init__() missing 1 required positional argument: 'loss'

In [22]:
import pandas as pd
import deepchem as dc
import tensorflow as tf

# GPU 메모리를 최대한 활용하도록 설정
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], False)  # 동적 메모리 할당 비활성화
        print("GPU set to use maximum memory")
    except RuntimeError as e:
        print(f"RuntimeError: {e}")
else:
    print("No GPU available")

RuntimeError: Physical devices cannot be modified after being initialized


In [23]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# 하이퍼파라미터 목록 정의
param_grid = {
    'graph_conv_layers': [[64, 64], [128, 128], [64, 128, 64]],
    'dense_layer_size': [128, 256],
    'dropout': [0.2, 0.3],
    'learning_rate': [1e-3, 5e-4, 1e-4],
    'batch_size': [256]  # GPU 메모리 상황에 따라 다양한 배치 크기를 실험
}

# 결과를 저장할 데이터프레임 초기화
results = []

# 하이퍼파라미터 조합 반복
from itertools import product
from tqdm import tqdm

combinations = list(product(
    param_grid['graph_conv_layers'],
    param_grid['dense_layer_size'],
    param_grid['dropout'],
    param_grid['learning_rate'],
    param_grid['batch_size']
))

for i, (graph_conv_layers, dense_layer_size, dropout, learning_rate, batch_size) in enumerate(tqdm(combinations, desc="Grid Search Progress")):
    
    print(f"\nRunning combination {i+1}/{len(combinations)}:")
    print(f"graph_conv_layers: {graph_conv_layers}, dense_layer_size: {dense_layer_size}, dropout: {dropout}, learning_rate: {learning_rate}, batch_size: {batch_size}")
    
    # 모델 초기화
    with tf.device('/GPU:0'):  # 명시적으로 GPU 사용 설정
        model = dc.models.GCNModel(
            n_tasks=1,
            graph_conv_layers=graph_conv_layers,
            dense_layer_size=dense_layer_size,
            dropout=dropout,
            mode='regression',
            learning_rate=learning_rate,
            batch_size=batch_size
        )

    # 모델 학습
    try:
        with tf.device('/GPU:0'):
            model.fit(train_dataset, nb_epoch=100)

        # 평가
        train_score = model.evaluate(train_dataset, [dc.metrics.Metric(dc.metrics.pearson_r2_score)])
        valid_score = model.evaluate(valid_dataset, [dc.metrics.Metric(dc.metrics.pearson_r2_score)])

        # 결과 저장
        results.append({
            'graph_conv_layers': graph_conv_layers,
            'dense_layer_size': dense_layer_size,
            'dropout': dropout,
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'train_r2_score': train_score['pearson_r2_score'],
            'valid_r2_score': valid_score['pearson_r2_score']
        })

        print(f"Train R2 Score: {train_score['pearson_r2_score']}, Validation R2 Score: {valid_score['pearson_r2_score']}")
    except Exception as e:
        print(f"Error during training: {e}")

# 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과 출력 및 저장
results_df = results_df.sort_values(by='valid_r2_score', ascending=False)
results_df.to_csv("gridsearch_results.csv", index=False)

# 결과 확인
print("\nTop Results:")
print(results_df.head())


Grid Search Progress:   0%|                                                                     | 0/36 [00:00<?, ?it/s]


Running combination 1/36:
graph_conv_layers: [64, 64], dense_layer_size: 128, dropout: 0.2, learning_rate: 0.001, batch_size: 256


In [19]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(train_dataset, [metric]))
print("Test set score:", model.evaluate(test_dataset, [metric]))

Training set score: {'pearson_r2_score': 0.8728071088137395}
Test set score: {'pearson_r2_score': 0.7761037587217018}


In [17]:
# 모델 평가 / GPT 파라미터 버전
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
train_score = model.evaluate(train_dataset, [metric])
valid_score = model.evaluate(valid_dataset, [metric])

print("Train score:", train_score)
print("Validation score:", valid_score)

Train score: {'pearson_r2_score': 0.8320443628967614}
Validation score: {'pearson_r2_score': 0.802325296546151}


In [40]:
from sklearn.metrics import r2_score

# 1. Test 데이터셋에 대한 예측값 추론
y_pred = model.predict(test_dataset)

# 2. 실제 y값 (test_dataset.y) 과 비교하여 R2 스코어 계산
test_r2 = r2_score(test_dataset.y, y_pred)

print("Test R2 Score:", test_r2)


Test R2 Score: 0.735293292904228


In [18]:
from sklearn.metrics import r2_score

# 1. Test 데이터셋에 대한 예측값 추론
y_pred = model.predict(test_dataset)

# 2. 실제 y값 (test_dataset.y) 과 비교하여 R2 스코어 계산
test_r2 = r2_score(test_dataset.y, y_pred)

print("Test R2 Score:", test_r2)


Test R2 Score: 0.7949052245292128


In [19]:
# 모델 훈련 후
model.save_checkpoint(model_dir="./Aquasol")  # 원하는 경로 지정

In [20]:
model